In [22]:
pip install mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 853.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 1.8 MB/s eta 0:00:00a 0:00:01
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
!python -m spacy download en_core_web_lg mistralai

     ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/400.7 MB 15.3 MB/s eta 0:00:21^C
     ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/400.7 MB 18.1 MB/s eta 0:00:18
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import kagglehub
import os

resumes_df_path = "data/UpdatedResumeDataSet.csv"
jobs_df_path = "data/data job posts.csv"

if not os.path.exists(resumes_df_path):
    print(f"Resumes dataset not found at {resumes_df_path}")
    print(f"Downloading from kaggle...")
    path = kagglehub.dataset_download("gauravduttakiit/resume-dataset")
    !mkdir -p data
    !mv $path/* ./data

if not os.path.exists(jobs_df_path):
    print(f"Jobs dataset not found at {jobs_df_path}")
    print(f"Downloading from kaggle...")
    path = kagglehub.dataset_download("madhab/jobposts")
    !mkdir -p data
    !mv $path/* ./data


resumes_df = pd.read_csv(resumes_df_path)
jobs_df = pd.read_csv(jobs_df_path)

/Users/andriipolishko/.pyenv/versions/3.10.15/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# TF-idf

In [9]:
resumes_df

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Example resume
resume_text = """
Experienced software engineer with expertise in Python, machine learning, and data analysis.
Proven ability to deliver high-quality solutions in fast-paced environments.
"""

resume_text_1 = resumes_df.loc[0]['Resume']
resume_text_3 = resumes_df.loc[961]['Resume']

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=10)

# Fit and transform the text
tfidf_matrix = vectorizer.fit_transform([resume_text_3])

# Get the feature names and their scores
feature_names = vectorizer.get_feature_names_out()
scores = tfidf_matrix.toarray()[0]

# Display results
important_words = sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)
print("Most Important Words:", important_words)

Most Important Words: [('windows', np.float64(0.595879571531124)), ('testing', np.float64(0.5107539184552492)), ('months', np.float64(0.34050261230349943)), ('exprience', np.float64(0.21281413268968716)), ('test', np.float64(0.21281413268968716)), ('web', np.float64(0.21281413268968716)), ('xp', np.float64(0.21281413268968716)), ('environment', np.float64(0.17025130615174972)), ('project', np.float64(0.17025130615174972)), ('role', np.float64(0.17025130615174972))]


In [20]:
import spacy
from collections import Counter
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

# Load English model
nlp = spacy.load("en_core_web_lg")

# Example resume
resume_text = """
Experienced software engineer with expertise in Python, machine learning, and data analysis.
Proven ability to deliver high-quality solutions in fast-paced environments.
"""

# Process text with spaCy
doc = nlp(resume_text_3)

# Extract candidate words (non-stop words and non-punctuation)
words = [token.text for token in doc if token.is_alpha and token.text.lower() not in STOP_WORDS]

# Calculate word frequencies
word_freq = Counter(words)

# Display the most common words
important_words = word_freq.most_common(10)
print("Most Important Words:", important_words)


Most Important Words: [('Windows', 10), ('Testing', 8), ('months', 8), ('Exprience', 5), ('Project', 4), ('Based', 4), ('Environment', 4), ('Role', 4), ('Duration', 4), ('Details', 3)]


# Mistral keyphrases getter

In [25]:
from mistralai import Mistral

model = "mistral-large-latest"

api_key = "JnaRE4F25Lq2cJLoVUmjPwjp9COhgY19"

client = Mistral(api_key)

In [32]:
resume_text_3

'Skill Set OS Windows XP/7/8/8.1/10 Database MYSQL, sql server 2005, 2008 & 2012 Languages Core Java Web Technology HTML, CSS Testing Manual Testing, Database Testing Other Bug tracking and reporting, End user handling.Education Details \r\nJanuary 2016 MCS  Pune, Maharashtra P.V.P College Pravaranagar\r\nJanuary 2011 HSC   A.K.Junior College\r\nJanuary 2009 SSC   A.K.Mahavidyalya\r\nJR TESTING ENGINEER \r\n\r\nJR TESTING ENGINEER - M-Tech Innovations Ltd\r\nSkill Details \r\nTESTING- Exprience - 24 months\r\nWINDOWS XP- Exprience - 24 months\r\nCSS- Exprience - 6 months\r\nDATABASE- Exprience - 6 months\r\nDATABASE TESTING- Exprience - 6 monthsCompany Details \r\ncompany - M-Tech Innovations Ltd\r\ndescription - Responsibilities â\x80¢     Analyzing the Testing Requirements â\x80¢     Preparing Test Plans, Test Scenarios â\x80¢     Preparing Test Cases â\x80¢     Executing the test cases â\x80¢     Tracking bugs â\x80¢     Coordinating developers in order to fix it â\x80¢     Preparin

In [31]:
chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "system",
            "content": "You are resume analyzer with a single role of extracting most important skills. Your output ALWAYS is following format: 'Most important skills: skill1, skill2, skill3, ...'",
        },
        {
            "role": "user",
            "content": resume_text_3
        }
    ]
)
print(chat_response.choices[0].message.content)

Most important skills: Testing, Windows OS (XP, 7, 8, 8.1, 10), Database Testing, Manual Testing, CSS, MYSQL, SQL Server (2005, 2008, 2012), Bug Tracking, Test Planning, Test Case Execution, Coordination with Developers, Reporting to Quality Manager.
